## Python Path
Before getting started it is important to add OpenVDS to the Python path. This can be done by installing OpenVDS as a Python Wheel or as a Python Egg. It's is possible to use a local build. When doing this it's important that the version of Python running the Jupyter Notebook kernels is the same version used to build OpenVDS. In Jupyter Notebook the About dialog under the Help menu will give you the Python version run by the Jupyter Notebook kernels.

For OpenVDS there is the OPENVDS_PYTHON_VERSION CMake variable that can be set to pick up a Python of desired version from the system.

The notebook will give errors if OpenVDS is not in the Python path.
The error will complain about ModuleNotFoundError:

`ModuleNotFoundError: No module named 'openvds'`

An example of how to import a local build on a Windows system would be to use the following two script lines: (Note that this is not needed if OpenVDS is in the PYTHONPATH environment variable or OpenVDS is installed as a Wheel or Egg).

In [ ]:
#This is not needed if OpenVDS is in the python path!
import sys
sys.path.append(r'C:\Projects\open-vds\out\build\x64-Debug\python')

Then OpenVDS can be imported

In [ ]:
import openvds
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.image as mpimg
import numpy as np
import time
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16, 16)

## Open a VDS connection and get the Layout object for the connection
Specific OpenOptions can to be used to open a connection to different cloud vendors. The AWS open options also relies on the AWS standardized credentials file in $HOME/.aws/credentials

Alternativly a url and connection string pair can be used. This scheme is descibed here:
http://osdu.pages.community.opengroup.org/platform/domain-data-mgmt-services/seismic/open-vds/connection.html

In [ ]:
#openOptions = openvds.AWSOpenOptions("bucket", "folder_in_bucket", "region")
#openOptions = openvds.AzureOpenOptions("connection_string", "container", "blob") #blob is ignored for now
#openOptions = openvds.GoogleOpenOptions("bucket", "folder_in_bucket")
#vds = openvds.open(openOptions)
vds = openvds.open("some url", "optional connection string")
layout = openvds.getLayout(vds)

## Layout
The layout can be used to retrieve properties about the VDS.

In [ ]:
print("ChannelCount: {}".format(layout.getChannelCount()))
for i in range(layout.getChannelCount()):
    print("  Channel Name: {}".format(layout.getChannelName(i)))
    print("    Value range: {} - {}".format(layout.getChannelValueRangeMin(i), layout.getChannelValueRangeMax(i)))
    
print("Dimensionality: {}".format(layout.getDimensionality()))
for i in range(layout.getDimensionality()):
    print("  Dimension name: {}".format(layout.getDimensionName(i)))
    print("    Number of samples: {}".format(layout.getDimensionNumSamples(i)))
    print("    Coordinate min max {} - {}".format(layout.getDimensionMin(i), layout.getDimensionMax(i)))

## Metadata accessor
Metadata can be accessed through the layout objects getMetadataXX function. The type is part of the key and no implicit conversion will occur.
- getMetadataInt
- getMetadataIntVector2
- getMetadataIntVector3
- getMetadataIntVector4
- getMetadataFloat
- getMetadataFloatVector2
- getMetadataFloatVector3
- getMetadataFloatVector4
- getMetadataDouble
- getMetadataDoubleVector2
- getMetadataDoubleVector3
- getMetadataDoubleVector4
- getMetadataString
- getMetadataBLOB




In [ ]:
MetaType = openvds.MetadataType
if layout.isMetadataDoubleVector2Available("SurveyCoordinateSystem", "Origin"):
    print("SurveyCoordinateSystem::Origin: {}".format(layout.getMetadataDoubleVector2("SurveyCoordinateSystem", "Origin")))
if layout.isMetadataDoubleVector2Available("SurveyCoordinateSystem", "InlineSpacing"):
    print("SurveyCoordinateSystem::InlineSpacing: {}".format(layout.getMetadataDoubleVector2("SurveyCoordinateSystem", "InlineSpacing")))
if layout.isMetadataDoubleVector2Available("SurveyCoordinateSystem", "CrosslineSpacing"):
    print("SurveyCoordinateSystem::CrosslineSpacing: {}".format(layout.getMetadataDoubleVector2("SurveyCoordinateSystem", "CrosslineSpacing")))
#layout.getMetadata("SEGY", "TextHeader", openvds.core.MetadataType.BLOB)
#layout.getMetadata("SEGY", "BinaryHeader", openvds.core.MetadataType.BLOB)

## The AccessManager api
The AccessManager api can be used to retieve data from the vds

In [ ]:
accessManager = openvds.VolumeDataAccessManager(vds)
axisDescriptors = [layout.getAxisDescriptor(dim) for dim in range(layout.getDimensionality())]
sliceType = 'inline'
sliceIndex = 55
sliceDimension = 2 if sliceType == 'inline' else 1 if sliceType == 'crossline' else 0 if sliceType == 'timeslice' else 0 if sliceType == 'depthslice' else -1

min = tuple(sliceIndex + 0 if dim == sliceDimension else 0 for dim in range(6))
max = tuple(sliceIndex + 1 if dim == sliceDimension else layout.getDimensionNumSamples(dim) for dim in range(6))

req = accessManager.requestVolumeSubset(min, max, format = openvds.VolumeDataChannelDescriptor.Format.Format_R32)
height = max[0] if sliceDimension != 0 else max[1]
width  = max[2] if sliceDimension != 2 else max[1]

data = req.data.reshape(width, height).transpose()
cmap=plt.get_cmap("seismic")
plt.set_cmap(cmap)
plt.imshow(data)
